In [1]:
import pandas as pd, glob, numpy as np
from scipy.spatial.distance import hamming
from myfuncs import deduplicate

In [2]:
paths = glob.glob("data/*_simhash.csv")
lst = []
for x in paths:
    df = pd.read_csv(x, index_col=0)
    lst +=  [df]
    
df = pd.concat(lst).reset_index(drop = True)

In [3]:
df

,fulltext_id,simhash,simhash_bit
0,15497914,3.416338e+18,0010111101101001010001010011100000010111010010...
1,9474991,1.756385e+19,1111001110111111010110001100101100101010010000...
2,15526043,8.011283e+18,0110111100101101110010110001100000010111010001...
3,10265653,3.719154e+17,0000010100101001010011110000110000011011011000...
4,3949397,4.465735e+18,0011110111111001011110100011000011000010010011...
...,...,...,...
195,9229598,1.682819e+19,1110100110001001101111101111110001100011110011...
196,12661472,1.778170e+19,1111011011000101010010010000010011111101100011...
197,51508201,1.093670e+18,0000111100101101011111110001111000010011010010...
198,407063,1.285711e+19,1011001001101101100111110001000011010010111010...


In [4]:
dct = {}
for i, x in zip(df.fulltext_id, df.simhash_bit):
    for j, y in zip(df.fulltext_id, df.simhash_bit):
       h = hamming(list(x), list(y)) * len(list(x))
       if i in dct.keys():           
           dct[i].update({j : h})
       else:
           dct[i] = {j : h}
         

In [5]:
x, y

('0110101100110001010010010111100000001111010010111100110100011001',
 '0110101100110001010010010111100000001111010010111100110100011001')

In [6]:
hamming(x, y)

0.0

In [8]:
ndf = pd.DataFrame(dct)

In [48]:
ndf.to_csv("data/simhash_distance.csv")

In [9]:
ndf2 = deduplicate(ndf)

In [12]:
ndf2.stack()

15497914  9474991     38.0
          15526043    15.0
          10265653    15.0
          3949397     24.0
          5618078     24.0
                      ... 
12661472  407063      28.0
          15941779    27.0
51508201  407063      28.0
          15941779    25.0
407063    15941779    35.0
Length: 19900, dtype: float64

In [15]:
v = ndf.stack().sort_values()

In [46]:
v

15497914  15497914     0.0
15845867  15845867     0.0
8529357   8529357      0.0
12284333  12284333     0.0
15836479  15836479     0.0
                      ... 
4498255   3384362     44.0
15419425  31035       44.0
31035     15419425    44.0
15762651  3949397     45.0
3949397   15762651    45.0
Length: 40000, dtype: float64

In [24]:
duplicates = v.loc[(v <= 3) & (v != 0)]

In [25]:
paths = glob.glob("data/*_fulltext.csv")
lst = []
for x in paths:
    df = pd.read_csv(x, index_col=0)
    lst +=  [df]
    
df = pd.concat(lst).reset_index(drop = True)

In [28]:
duplicates.reset_index().level_0

0    9008985
1    9229598
2    9841568
3    9372822
4    8529357
5    9229598
6    9841568
7    8529357
8    9474991
9    9008985
Name: level_0, dtype: int64

In [37]:
duplicates

9008985  9229598    1.0
9229598  9008985    1.0
9841568  9372822    1.0
9372822  9841568    1.0
8529357  9229598    2.0
9229598  8529357    2.0
9841568  9474991    3.0
8529357  9008985    3.0
9474991  9841568    3.0
9008985  8529357    3.0
dtype: float64

In [32]:
dupli_texts = df.loc[df.fulltext_id.isin(duplicates.reset_index().level_0)]

In [44]:
df.loc[df.fulltext_id == 9474991].fulltext.values[0]

'Om "Landbruket på Innherred"\n"Landbruket på Innherred" er eit prosjekt der "utviklingen i omfang og betydning av landbruket i regionen synliggjøres, blant annet i forhold til antall gårdsbruk, produksjonsomfang, antall sysselsatte/årsverk, verdiskaping og ringvirkninger i ulike produksjoner. Det har også vært et ønske å synliggjøre skogbrukets betydning, og betydning av andre landbruksbaserte næringer (tilleggsnæringer)."\nKommunal forvaltning, tillitsvalde i landbruksnæringa og andre aktørar knytta til næringa har behov for dokumentasjon som kan synleggjera landbruket har å seie, situasjon og utviklingstrekk på kommunalt og regionalt nivå. Det er behov for dokumentasjon overfor\nkommunal forvaltning\nlandbruket sine tillitsvalde, som møter politikarar, ungdom (i forhold til rekrutteringsarbeid), bønder (for å setja fokus på samanhengane langs verdikjeder og for å motivera til auka verdiskaping)\nandre næringar\nLandbruksforum for Innherredskommunane ynsker òg å setja fokus på felles

In [45]:
df.loc[df.fulltext_id == 9841568].fulltext.values[0]

'"Landbruket på Innherred" er eit prosjekt der "utviklingen i omfang og betydning av landbruket i regionen synliggjøres, blant annet i forhold til antall gårdsbruk, produksjonsomfang, antall sysselsatte/årsverk, verdiskaping og ringvirkninger i ulike produksjoner. Det har også vært et ønske å synliggjøre skogbrukets betydning, og betydning av andre landbruksbaserte næringer (tilleggsnæringer)."\nFoto toppbanner: Eva Pauline Hedegart\nDette er utskriftsversjon av https://www.landbruksforum.no/innherred/info/index.php?lang=nyn&viskommune=inder%C3%83%C2%83%C3%82%C2%83%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2%82%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2%83%C3%83%C2%82%C3%82%C2%82%C3%83%C2%83%C3%82%C2%82%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2%83%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2%82%C3%83%C2%82%C3%82%C2%82%C3%83%C2%83%C3%82%C2%83%C3%83%C2%82%C3%82%C2%82%C3%83%C2%83%C3%82%C2%82%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2%83%C3%83%C2%82%C3%82%C2%83%C3%83%C2%83%C3%82%C2

In [43]:
dupli_texts

,fulltext_id,hash,fulltext
34,9229598,f1731238982eede2eb4fcc7b59fa1c64663edd77,Begrepsavklaring og avgrensing\nJordbruk\nJord...
36,9841568,72240c05d7bc9cee6017f6e559178f3ab2e1d293,"""Landbruket på Innherred"" er eit prosjekt der ..."
138,8529357,e78c50c092d84416bcd5fde8000c200dfcecd3bf,Begrepsavklaring og avgrensing\nJordbruk\nJord...
144,9008985,ca8fce5fb18150d0bd405af76fd066854de1b8dd,Begrepsavklaring og avgrensing\nJordbruk\nJord...
147,9372822,a980a7ffb70783f6e18b95bee342646772f30f88,Verdal kommune Innbyggertall 14.162 * Flateare...
148,9474991,99da02dbadaf8fcd49f5e5bca1f56ebf663b1069,"Om ""Landbruket på Innherred""\n""Landbruket på I..."


In [36]:
for x in dupli_texts.fulltext:
    print(x, "\n\n")

Begrepsavklaring og avgrensing
Jordbruk
Jordbruk defineres som tradisjonell produksjon av jord- og hagebruksråvarer som mjølk, kjøt, ull, honning, egg, korn, oljefrø, erter, frukt, bær, poteter, grønnsaker, blomedyrking og andre jordbruksproduksjonar (pelsdyr, hjorteoppdrett, birøkt med meir). Det handlar her om tradisjonelt husdyrhald og planteproduksjon (dette femnar òg utleige av jordbruksareal til jordbruksaktivitet og utleige av maskiner i mindre målestokk).
Tilleggsnæringar
Næringsaktivitet på gardsbruk/landbrukseigedom som vert driven i tillegg til tradisjonelt landbruk, der ein eller flere av bruket sine ressursar (areal, bygningar, maskiner og landbruksråvarer) høyrer med som faktorinnsats i produksjonen.
Produksjonsverdi (omsetning)
Summen av salsinntekter (markedsinntekter) og produksjonstilskot
Verdiskaping
Fleire mål kan nyttast for å kasta lys over verdiskapinga i landbruksrelatert næringsvverksemd. Vi har her definert verdiskapinga som nettoprodukt, det vil sei det som e